# 빅분기Part5-작업2유형-분석연습
p.389~ 

### 문제1
고객 3500명에 대한 학습용 데이터(x_train, y_train)을 이용하여 성별 예측 모형을 만든 후, 이를 평가용 데이터(x_test)에 적용하여 얻은 2482명 고객의 성별 예측값(남자일 확률)을 다음과 같은 형식의 csv 파일로 생성하시오 (제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점)
- y_train: 고객의 성별 데이터. 학습 데이터. 3500명
- x_train, x_test: 고객의 상품 구매 속성. 학습 및 평가용

##### 제출 형식
- custid, gender
- 3500, 0.267
- 3501, 0.578
- 3502, 0.885

- 22.6.19.일 22:22 데이터 로드만.
- 22.6.20.월 14:30~16:57 predict 에서 Memory부족 에러나고 막힘. 8GB 넘게 먹어서 이 놋북에서 실행불가. 혹은 램 덜 먹게 조치??
> n_estimators= 10 으로 해결
- 22.6.20.월 18:00~22:30 ValueError: could not broadcast input array from shape (3500,3500) into shape (3500,) 및 #ValueError: multiclass-multioutput format is not supported
> y_test_proba 의 형태가 (2,3500,3500) 꼴로 나온다. 왜지

y_train에 index가 포함된 경우
#### predict 는 슬라이싱 하지 말아야 하고
- model.fit(x_train, y_train) #display(pd.DataFrame(y_train_predict).head()) # <<< 2열 (인덱스,예측 결과값)
- y_test_predict= model.predict(x_test) 

#### proba 는 슬라이싱 해서 gender 만 내도록 해야 한다.
- model.fit(x_train, y_train.iloc[:,1]) # <<< 없는 cust_id 가 방해가 되나 싶어서 1열만 내도록 학습
- y_test_proba= model.predict_proba(x_test) # <<< train 으로 만든 모델에 x_test 넣기. 결과 제출용으로 필수

#### 애초에 y_train 에서 index 부분은 drop 하고 따로 저장했어야?

##### error 회피하려다 초가삼간 태운다
그렇게 추정한 각 정보는 누구 id 것인데?
> id 붙이면 해결!. train 으로 model 만들적부터 오름차순으로만 되어있으면 가능

### 주석 싹싹 지우고 다시2
22.6.21 15:44/ y_train 에서 id 빼고 학습시켜야 한다. concat 은 Series 끼리만 []로 가능

In [88]:
## 데이터 로드, null, dtypes 체크 #이상치 체크, 인코딩, 파생변수, 상관관계, 변수 드랍, x_test에도 전처리 
## 스케일링 # 모델링, train score, test score # id붙이기, to_csv

## gender 일 확률. proba
import time
start_time= time.time()
import pandas as pd
pd.set_option('display.max_columns',None)
x_train= pd.read_csv('bigData/x_train.csv', encoding='CP949')
y_train= pd.read_csv('bigData/y_train.csv', encoding='CP949')
x_test= pd.read_csv('bigData/x_test.csv', encoding='CP949') # Wow! x_test 넣어야할 것을 x_train 넣었었다.

## null 처리. 환불 null 은 환불이 없었다고 가정. fillna(0)
x_train['환불금액']= x_train['환불금액'].fillna(0)
x_test['환불금액']= x_test['환불금액'].fillna(0)

## dtypes 체크 - 숫자인데 문자로 되어있는 건 아니 보인다

## 이상치 보기. 구매액이 매우 크다고 해서 절삭하는 건 말이 안 된다고 본다.
def outlierCheck(data):
    dataNum= data.loc[:,data.dtypes!=object] # 숫자만
    desc= dataNum.describe()
    min1= desc.loc['min']
    max1= desc.loc['max']
    std= desc.loc['std']
    mean= desc.loc['mean']
    maxBoundary= mean+1.5*std
    minBoundary= mean-1.5*std
    
    return minBoundary, maxBoundary

minB, maxB= outlierCheck(x_train)
minB_test, maxB_test= outlierCheck(x_test)

## 파생변수. 주말방문 경험 유무? 1회 최대구매액 구간- boundary 밖 & median? 환불여부?
conditionWeekend= x_train['주말방문비율']>0
x_train.loc[conditionWeekend, '주말방문여부']= 1
x_train.loc[~conditionWeekend, '주말방문여부']= 0

conditionRefund= x_train['환불금액']>0
x_train.loc[conditionRefund, '환불여부']= 1
x_train.loc[~conditionRefund, '환불여부']= 0

conditionMaxBuyOutlier= x_train['최대구매액']> maxB['최대구매액']
conditionMaxBuy= (x_train['최대구매액']<= maxB['최대구매액']) & (x_train['최대구매액'] > x_train['최대구매액'].median())
conditionMaxBuySmall= x_train['최대구매액'] <= x_train['최대구매액'].median()
x_train.loc[conditionMaxBuyOutlier, '최대구매액많은편']=2
x_train.loc[conditionMaxBuy, '최대구매액많은편']=1
x_train.loc[conditionMaxBuySmall, '최대구매액많은편']=0

## 파생변수- x_test
conditionWeekend= x_test['주말방문비율']>0
x_test.loc[conditionWeekend, '주말방문여부']= 1
x_test.loc[~conditionWeekend, '주말방문여부']= 0

conditionRefund= x_test['환불금액']>0
x_test.loc[conditionRefund, '환불여부']= 1
x_test.loc[~conditionRefund, '환불여부']= 0

conditionMaxBuyOutlier= x_test['최대구매액']> maxB_test['최대구매액']
conditionMaxBuy= (x_test['최대구매액']<= maxB_test['최대구매액']) & (x_test['최대구매액'] > x_test['최대구매액'].median())
conditionMaxBuySmall= x_test['최대구매액'] <= x_test['최대구매액'].median()
x_test.loc[conditionMaxBuyOutlier, '최대구매액많은편']=2
x_test.loc[conditionMaxBuy, '최대구매액많은편']=1
x_test.loc[conditionMaxBuySmall, '최대구매액많은편']=0

## 메모리 관리. 변수 지우기
x_train.drop(columns='환불금액', inplace=True)
x_test.drop(columns='환불금액', inplace=True)

## 책- 상관관계
x_train.drop(columns='최대구매액', inplace=True)
x_test.drop(columns='최대구매액', inplace=True)

## 인코딩. 범주화
from sklearn.preprocessing import LabelEncoder
encoder= LabelEncoder()
x_train['주구매상품']= encoder.fit_transform(x_train['주구매상품']) # astype 바로 못 붙인다
x_train['주구매지점']= encoder.fit_transform(x_train['주구매지점'])
x_train['주구매상품']= x_train['주구매상품'].astype('category') # inplace 없다
x_train['주구매지점']= x_train['주구매지점'].astype('category')
x_train['주말방문여부']= x_train['주말방문여부'].astype('category')
x_train['환불여부']= x_train['환불여부'].astype('category')
x_train['최대구매액많은편']= x_train['최대구매액많은편'].astype('category')

## 인코딩. 범주화- x_test
x_test['주구매상품']= encoder.fit_transform(x_test['주구매상품'])
x_test['주구매지점']= encoder.fit_transform(x_test['주구매지점'])
x_test['주구매상품']= x_test['주구매상품'].astype('category')
x_test['주구매지점']= x_test['주구매지점'].astype('category')
x_test['주말방문여부']= x_test['주말방문여부'].astype('category')
x_test['환불여부']= x_test['환불여부'].astype('category')
x_test['최대구매액많은편']= x_test['최대구매액많은편'].astype('category')


## 그냥 drop 시키고 cust_id 만 따로 저장
x_train_custid= x_train['cust_id']
x_train.drop(columns='cust_id', inplace=True)

x_test_custid= x_test['cust_id']
x_test.drop(columns='cust_id', inplace=True)

## 스케일링 - 시험때는 조건말고 눈으로 보고 컬럼 골라내서 for 문 돌리자
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()
x_trainNumCols= x_train.columns[(x_train.dtypes!=object) &(x_train.dtypes!='category')]
bigNumCols= x_trainNumCols[x_train.describe().loc['max']> 30] # 30 근거: 범주컬럼 냅두고 싶어서 눈으로 정한 값
for col in bigNumCols:
    x_train[col]= scaler.fit_transform(x_train[[col]])
    
## 스케일링 - x_test
x_testNumCols= x_test.columns[(x_test.dtypes!=object) &(x_test.dtypes!='category')]
bigNumCols= x_testNumCols[x_train.describe().loc['max']> 30] # 30 근거: 범주컬럼 냅두고 싶어서 눈으로 정한 값
for col in bigNumCols:
    x_test[col]= scaler.fit_transform(x_test[[col]])
    
beforeModeling= time.time()
print(f"모델링전까지 수행시간: {beforeModeling-start_time:.6} sec")

## 모델링
from sklearn.ensemble import RandomForestClassifier
## 메모리 이슈 -  여분이 7.9GB 인데 랜포 40으로 돌리면 8.5GB 를 드신다. 랜포말고 DecisionTree는?
model= RandomForestClassifier(n_estimators= 10) # help(model) #fit # 메모리 줄이기. n_estimators=100 default. 10으로 해결.


## id 가 포함된 채로 y_train 목표로 학습(fit)시킬 경우, y_test_predict 의 id 에는 중복값이 들어가있다. y_test_proba는 (2,2438,3500)이 되고.
model.fit(x_train, y_train.iloc[:,1]) # y_test_predict 용 모델도 gender 만 넣고
y_test_predict= model.predict(x_test) 
y_test_proba= model.predict_proba(x_test) # <<< 결과 제출용


## help(pd.concat) # pd.concat([series1, series2], ignore_index=True, axis=1) # y_test_predict 는 ndarray
# display(pd.concat([x_test_custid, pd.Series(y_test_predict)], ignore_index=True, axis=1)) 

## x_test_predict 와 x_test_proba 를 비교하여, proba 의 어느 열을 남길지 결정
## > predict 값이 1(남자) 인 게, proba의 2번째 컬럼[1]
## 데이터 설명에서 gender==1 이 남자.

# print(f"y_test_proba {y_test_proba} \ny_test_proba[0] {y_test_proba[0]}")
# display(pd.DataFrame(y_test_proba).head()) # <<< columns=[0,1], 첫 행= [0.5, 0.5]
# display(pd.DataFrame(y_test_proba).info())
# display(pd.concat([ x_test_custid, pd.DataFrame(y_test_proba).iloc[:,1] ], axis=1).rename(columns={"cust_id":"custid", 1:"gender"}))
genderManProba= pd.concat([ x_test_custid, pd.DataFrame(y_test_proba).iloc[:,1] ], axis=1).rename(columns={"cust_id":"custid", 1:"gender"})
# print(y_train)

# help(pd.DataFrame.to_csv) # index
genderManProba.to_csv('data/김형준-1번-proba.csv', index=False)

## score 는 필수사항은 아니므로 일단 보류
from sklearn.metrics import roc_auc_score

afterModeling= time.time()
print(f"모델링 시간: {afterModeling-beforeModeling:.6} sec")

모델링전까지 수행시간: 0.214001 sec
모델링 시간: 0.260998 sec


In [74]:
# help(pd.concat) # pd.concat([s1, s2], ignore_index=True, axis=1)

In [90]:
###############################################################################

### 주석 싹 지우고 다시
22.6.20 21:00 / x_test 에도 전처리 적용해야 한다

In [76]:
## 데이터 로드, null, dtypes 체크 #이상치 체크, 인코딩, 파생변수 # 스케일링 # 모델링, train score, test score # 형식에 맞게 출력

## gender 일 확률. proba
import time
start_time= time.time()
import pandas as pd
pd.set_option('display.max_columns',None)
x_train= pd.read_csv('bigData/x_train.csv', encoding='CP949')
y_train= pd.read_csv('bigData/y_train.csv', encoding='CP949')
x_test= pd.read_csv('bigData/x_test.csv', encoding='CP949') # Wow! x_test 넣어야할 것을 x_train 넣었었다.

## null 처리. 환불 null 은 환불이 없었다고 가정. fillna(0)
x_train['환불금액']= x_train['환불금액'].fillna(0)
x_test['환불금액']= x_test['환불금액'].fillna(0)

## dtypes 체크 - 숫자인데 문자로 되어있는 건 아니 보인다

## 이상치 보기. 구매액이 매우 크다고 해서 절삭하는 건 말이 안 된다고 본다.
def outlierCheck(data):
    dataNum= data.loc[:,data.dtypes!=object] # 숫자만
    desc= dataNum.describe()
    min1= desc.loc['min']
    max1= desc.loc['max']
    std= desc.loc['std']
    mean= desc.loc['mean']
    maxBoundary= mean+1.5*std
    minBoundary= mean-1.5*std
    
    return minBoundary, maxBoundary

minB, maxB= outlierCheck(x_train)
minB_test, maxB_test= outlierCheck(x_test)

## 파생변수. 주말방문 경험 유무? 1회 최대구매액 구간- boundary 밖 & median? 환불여부?
conditionWeekend= x_train['주말방문비율']>0
x_train.loc[conditionWeekend, '주말방문여부']= 1
x_train.loc[~conditionWeekend, '주말방문여부']= 0

conditionRefund= x_train['환불금액']>0
x_train.loc[conditionRefund, '환불여부']= 1
x_train.loc[~conditionRefund, '환불여부']= 0

conditionMaxBuyOutlier= x_train['최대구매액']> maxB['최대구매액']
conditionMaxBuy= (x_train['최대구매액']<= maxB['최대구매액']) & (x_train['최대구매액'] > x_train['최대구매액'].median())
conditionMaxBuySmall= x_train['최대구매액'] <= x_train['최대구매액'].median()
x_train.loc[conditionMaxBuyOutlier, '최대구매액많은편']=2
x_train.loc[conditionMaxBuy, '최대구매액많은편']=1
x_train.loc[conditionMaxBuySmall, '최대구매액많은편']=0

## 파생변수- x_test
conditionWeekend= x_test['주말방문비율']>0
x_test.loc[conditionWeekend, '주말방문여부']= 1
x_test.loc[~conditionWeekend, '주말방문여부']= 0

conditionRefund= x_test['환불금액']>0
x_test.loc[conditionRefund, '환불여부']= 1
x_test.loc[~conditionRefund, '환불여부']= 0

conditionMaxBuyOutlier= x_test['최대구매액']> maxB_test['최대구매액']
conditionMaxBuy= (x_test['최대구매액']<= maxB_test['최대구매액']) & (x_test['최대구매액'] > x_test['최대구매액'].median())
conditionMaxBuySmall= x_test['최대구매액'] <= x_test['최대구매액'].median()
x_test.loc[conditionMaxBuyOutlier, '최대구매액많은편']=2
x_test.loc[conditionMaxBuy, '최대구매액많은편']=1
x_test.loc[conditionMaxBuySmall, '최대구매액많은편']=0

## 메모리 관리. 변수 지우기
x_train.drop(columns='환불금액', inplace=True)
x_test.drop(columns='환불금액', inplace=True)

## 책- 상관관계
x_train.drop(columns='최대구매액', inplace=True)
x_test.drop(columns='최대구매액', inplace=True)

## 인코딩. 범주화
from sklearn.preprocessing import LabelEncoder
encoder= LabelEncoder()
x_train['주구매상품']= encoder.fit_transform(x_train['주구매상품']) # astype 바로 못 붙인다
x_train['주구매지점']= encoder.fit_transform(x_train['주구매지점'])
x_train['주구매상품']= x_train['주구매상품'].astype('category') # inplace 없다
x_train['주구매지점']= x_train['주구매지점'].astype('category')
x_train['주말방문여부']= x_train['주말방문여부'].astype('category')
x_train['환불여부']= x_train['환불여부'].astype('category')
x_train['최대구매액많은편']= x_train['최대구매액많은편'].astype('category')

## 인코딩. 범주화- x_test
x_test['주구매상품']= encoder.fit_transform(x_test['주구매상품'])
x_test['주구매지점']= encoder.fit_transform(x_test['주구매지점'])
x_test['주구매상품']= x_test['주구매상품'].astype('category')
x_test['주구매지점']= x_test['주구매지점'].astype('category')
x_test['주말방문여부']= x_test['주말방문여부'].astype('category')
x_test['환불여부']= x_test['환불여부'].astype('category')
x_test['최대구매액많은편']= x_test['최대구매액많은편'].astype('category')


## 그냥 drop 시키고 cust_id 만 따로 저장( 'cust_id 를 index 로 만들' - 이 방법이 좀 문제를 일으키는 것 같아서 )
# train 에 index 떼고 y_train 을 id 째 넣어도 여전히 안된다
# 나중에 되돌리는 건 df.reset_index(drop=False, inplace=True)
x_train_custid= x_train['cust_id'] # x_train.index= x_train['cust_id']
x_train.drop(columns='cust_id', inplace=True)

x_test_custid= x_test['cust_id'] #x_test.index= x_test['cust_id']
x_test.drop(columns='cust_id', inplace=True)

## 스케일링 - 시험때는 조건말고 눈으로 보고 컬럼 골라내서 for 문 돌리자
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()
x_trainNumCols= x_train.columns[(x_train.dtypes!=object) &(x_train.dtypes!='category')]
bigNumCols= x_trainNumCols[x_train.describe().loc['max']> 30] # 30 근거: 범주컬럼 냅두고 싶어서 눈으로 정한 값
for col in bigNumCols:
    x_train[col]= scaler.fit_transform(x_train[[col]])
    
## 스케일링 - x_test
x_testNumCols= x_test.columns[(x_test.dtypes!=object) &(x_test.dtypes!='category')]
bigNumCols= x_testNumCols[x_train.describe().loc['max']> 30] # 30 근거: 범주컬럼 냅두고 싶어서 눈으로 정한 값
for col in bigNumCols:
    x_test[col]= scaler.fit_transform(x_test[[col]])
    
beforeModeling= time.time()
print(f"모델링전까지 수행시간: {beforeModeling-start_time:.6} sec")

## 모델링
from sklearn.ensemble import RandomForestClassifier
model= RandomForestClassifier(n_estimators= 10) # help(model) #fit # 메모리 줄이기. n_estimators=100 default. 10으로 해결.

# display(x_train.head())
# print(x_train.shape, x_test.shape) # 둘다 (3500,10). x_test 가 왜 3500? 원래 (2482,10) 인데
# print(x_train.columns, x_test.columns) # 컬럼 일치
# print("y_train.shape: ", y_train.shape) #(3500, 2) . 정상적으로 2컬럼.
# print("y_train.iloc[:,1]: ", y_train.iloc[:,1])


## 메모리 이슈 -  여분이 7.9GB 인데 랜포 돌리면 8.5GB 를 드신다. PC에서 돌리면 돌아갈까? 랜포말고 DecisionTree는?
# n_estimators=10 으로 해결. # n_estimators=40 에서 재발- MemoryError: could not allocate 458752000 bytes. 437MB

## train 에 index 넣는 게 좀 문제 있어 보여서 cust_id 그냥 drop 시키고 진행
## predict 는 슬라이싱 하지 말아야 하고
# id 가 꼬이게 된다. model.fit(x_train, y_train) #display(pd.DataFrame(y_train_predict).head()) # <<< 2열 (인덱스,예측 결과값)
model.fit(x_train, y_train.iloc[:,1]) # y_test_predict 용 모델도 gender 만 넣고
y_test_predict= model.predict(x_test) 

## proba 는 슬라이싱 해서 gender 만 내도록 해야 한다.
model.fit(x_train, y_train.iloc[:,1]) # <<< 없는 cust_id 가 방해가 되나 싶어서 1열만 내도록 학습
y_test_proba= model.predict_proba(x_test) # <<< train 으로 만든 모델에 x_test 넣기. 결과 제출용으로 필수

# y_train_predict= model.predict(x_train) # <<< score 는 보류

# display(pd.DataFrame(y_test_predict).head())
## help(pd.concat) # pd.concat([series1, series2], ignore_index=True, axis=1)
display(pd.concat([x_test_custid, pd.Series(y_test_predict)], ignore_index=True, axis=1)) # y_test_predict 는 ndarray

## id 가 포함된 채로 y_train 목표로 학습(fit)시킬 경우, y_test_predict 의 id 에는 중복값이 들어가있다.
# display(pd.DataFrame(y_test_predict).sort_values(0, ascending=True).head())

# print(f"len(y_test_predict) {len(y_test_predict)}")
# print(f"y_test_predict {y_test_predict}")
print(f"y_test_proba {y_test_proba} \ny_test_proba[0] {y_test_proba[0]}")
# print(f"len(y_test_proba) {len(y_test_proba)} \nlen(y_test_proba[0]) {len(y_test_proba[0])}")
# print(f"y_test_proba {y_test_proba} \ny_test_proba[0] {y_test_proba[0]} \ny_test_proba[0][0] {y_test_proba[0][0]}")

## y_train 에 index가 있으면 빼고 학습시켜야 proba가 정상적으로 나온다
# 해결! ValueError: could not broadcast input array from shape (3500,3500) into shape (3500,)
display(pd.DataFrame(y_test_proba).head()) # <<< columns=[0,1], 첫 행= [0.5, 0.5]
display(pd.DataFrame(y_test_proba).info())

# print(f"y_test_proba {y_test_proba} \ny_test_proba[0] {y_test_proba[0]} \ny_test_proba[0][0]\
#     {y_test_proba[0][0]}")

# print(f"len(y_test_proba) {len(y_test_proba)} \nlen(y_test_proba[0]) {len(y_test_proba[0])}  \
#     \nlen(y_test_proba[0][0]) {len(y_test_proba[0][0])}")

# # help(pd.DataFrame) # example. df2 = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]), columns=['a', 'b', 'c'])
# print(pd.DataFrame(data=y_test_proba, columns=['custid', 'gender'])) 


## score 는 필수사항은 아니므로 일단 보류
from sklearn.metrics import roc_auc_score
# y_test 는 애초에 없다. print(roc_auc_score(y_test, y_test_predict))
#ValueError: multiclass-multioutput format is not supported. 보류
# 내가 만든 모델(y_train_predict)의 점수만 확인 가능. 
# print(roc_auc_score(y_train, pd.DataFrame(data=y_train_predict, columns=['cust_id', 'gender'])) )


# afterModeling= time.time()
# print(f"모델링 시간: {afterModeling-beforeModeling:.6} sec")

모델링전까지 수행시간: 0.181998 sec


,0,1
0,3500,1
1,3501,0
2,3502,0
3,3503,0
4,3504,0
...,...,...
2477,5977,1
2478,5978,0
2479,5979,0
2480,5980,0


y_test_proba [[0.6 0.4]
 [0.2 0.8]
 [0.4 0.6]
 ...
 [0.4 0.6]
 [0.6 0.4]
 [0.8 0.2]] 
y_test_proba[0] [0.6 0.4]


,0,1
0,0.6,0.4
1,0.2,0.8
2,0.4,0.6
3,0.6,0.4
4,0.5,0.5


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2482 entries, 0 to 2481
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       2482 non-null   float64
 1   1       2482 non-null   float64
dtypes: float64(2)
memory usage: 38.9 KB


None

### 각종 시행착오 및 주석 포함

1. loc에 컬럼 여럿 print(x_train.describe().loc[:,('총구매액', '환불금액')])
2. print 대신 display 로 보면 값-컬럼 정렬되있어서 보기 좋다
3. [column rename python](https://rfriend.tistory.com/468) 특정 컬럼만 콕 집어 바꿀 수 있다.  df.rename(columns = {"old": "new"}, inplace = True)
4. 메모리 부족 에러: 코랩이든 남의 컴이든 똑같을 것. 변수 개수 줄이고 각종 중간 변수들 생략하고, 에러나도 메모리 해제하게끔 조치해보자. 
> n_estimators=100 대신 40 넣고, OS 상의 다른 서비스 끄고, 변수 줄여서 성공. 파이썬에 메모리 해제하는 코드는 없대
4. 랜덤포레스트 에서 model.score() 시, "ValueError: X has 2 features, but DecisionTreeClassifier is expecting 10 features as input"
> 랜포에 왠 Decision.. 일단 따지지 않고 model.score() 안 쓰고 sklearn.metrics: roc_auc_score 쓰는걸로 임시방편
5. [ValueError: could not convert string to float: '기타'](https://hashcode.co.kr/questions/9650/valueerror-could-not-convert-string-to-float-%EB%AC%B8%EC%9E%90-%EB%9D%BC%EB%8A%94-%EC%97%90%EB%9F%AC). 기타 컬럼의 string 에 숫자 연산이 필요한 무언가를 했을 것
> x_train, y_train, x_test 에 모두 '기타' 컬럼이 존재하지 않는다. 아마 LabelEncoding 이 필요한 '주구매상품, 주구매지점' 에 연산을 하다가 가장 처음 만난 요소가 '주구매상품:기타' 일 것.
- x_test 에도 x_train 에 했던 것을 똑같이 해줘야 한다는 건데.. x_validation 넣으면 또 안될거 아녀
> 시험 혹은 채점자의 검증은 y_test 와 일치여부만을 본다. x_validation 이 만약에 있다면 그 또한 전처리해서 모델러가 사용할 데이터

In [14]:
# ?????? '기타' 라는 컬럼이 없는데 어디서 나온 컬럼이여-
# print(x_train.columns, x_test.columns, y_train.columns)
# data= pd.read_csv('bigData/x_train.csv', encoding='CP949')
# data['주구매상품'].value_counts() # 여기에 '기타' 있다. 가장 많은 유형

In [6]:
## 데이터 로드, null, dtypes 체크 #이상치 체크, 인코딩, 파생변수 # 스케일링 # 모델링, train score, test score # 형식에 맞게 출력
## gender 일 확률. proba


# !ls
import time
start_time= time.time()
import pandas as pd
pd.set_option('display.max_columns',None)

# help(pd.read_csv) # encoding #UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc3 in position 8: invalid continuation byte
x_train= pd.read_csv('bigData/x_train.csv', encoding='CP949')
y_train= pd.read_csv('bigData/y_train.csv', encoding='CP949')
x_test= pd.read_csv('bigData/x_train.csv', encoding='CP949')
# display(type(x_train))
# display(x_train.shape, x_train.info(). x_train.head(2)) # AttributeError: 'NoneType' object has no attribute 'x_train'
# info() 뒤에 , 대신 . 을 찍었다
# display(x_train.shape, x_train.info(), x_train.head(2)) # 총 3500개. 환불금액 1205개. 주구매상품 주구매지점 object
# display(y_train.shape, y_train.info(), y_train.head(2)) # cust_id, gender. 제출형식에는 _ 없음에 유의
# display(x_test.shape, x_test.info(), x_test.head(2)) # 총 3500개. train-test 가 반반이네.

## null 처리. 환불 null 은 환불이 없었다고 가정. fillna(0)
# display(x_train['환불금액'].min() ,'---') # min: 5600. null 이 아닌 것 중 0인 것은 없다
# 틀림 x_train.loc[x_train.isnull().sum()!=0]
# display(x_train.loc[x_train.isnull().sum(1) !=0],'---')
# display(x_train.describe().loc[:,('총구매액', '환불금액')],'---')
# 환불금액이 null 인 것의 의미? 가정1. 적지 않았다. 가정2. 환불이 없었다. - 2295 개나 적지 않았다는걸 받아들이기 어려움
# 환불금액을 mean 값으로 대체할 경우 문제점: 총구매액보다 커질 수 있다.
x_train['환불금액']= x_train['환불금액'].fillna(0)
# display(x_train.loc[x_train.isnull().sum(1)!=0].describe(),'---')
# display(y_train.loc[x_train.isnull().sum(1)!=0],'---') # y_train 에는 null 이 없다

## dtypes 체크 - 숫자인데 문자로 되어있는 건 아니 보인다
# display(x_train.columns[x_train.dtypes==object])
# display(x_train.loc[:,x_train.dtypes==object].iloc[:,0].value_counts().count()) # 42종 카테고리
# display(x_train.loc[:,x_train.dtypes==object].iloc[:,1].value_counts().count()) # 24종 카테고리
# display(x_train.columns[x_train.dtypes!=object])
# display(x_train.loc[:,x_train.dtypes!=object])
# display(x_train.describe())

## 이상치 보기. 구매액이 매우 크다고 해서 절삭하는 건 말이 안 된다고 본다.
# 대신 boundary 바깥의 값을 파생변수에 이용할 수도?
def outlierCheck(data):
    dataNum= data.loc[:,data.dtypes!=object] # 숫자만
    
    desc= dataNum.describe()
    min1= desc.loc['min']
    max1= desc.loc['max']
    std= desc.loc['std']
    mean= desc.loc['mean']
    maxBoundary= mean+1.5*std
    minBoundary= mean-1.5*std
#     print("maxBoundary", maxBoundary, '---')
#     print("minBoundary", minBoundary, '---')
    
#     print((dataNum>maxBoundary).sum(0), '---') # 큰 이상치 수백개
#     print((dataNum<minBoundary).sum(0), '---') # 작은 이상치 없음
    
    return minBoundary, maxBoundary

minB, maxB= outlierCheck(x_train)

## 파생변수. 주말방문 경험 유무? 1회 최대구매액 구간- boundary 밖 & median? 환불여부?
# 주말방문여부, 환불여부, 최대구매액많은편
# display(x_train.head(3))
conditionWeekend= x_train['주말방문비율']>0
x_train.loc[conditionWeekend, '주말방문여부']= 1
x_train.loc[~conditionWeekend, '주말방문여부']= 0
# display(x_train['내점일수'].value_counts(), '---')
# display(x_train['구매주기'].count(), '---')
# display(x_train[x_train['구매주기']<=4].count(), '---') # 기준값 모르겠다
# display(x_train.describe().loc[:,('총구매액','최대구매액','환불금액')], '---')
conditionRefund= x_train['환불금액']>0
x_train.loc[conditionRefund, '환불여부']= 1
x_train.loc[~conditionRefund, '환불여부']= 0
# print(maxB)
conditionMaxBuyOutlier= x_train['최대구매액']> maxB['최대구매액']
conditionMaxBuy= (x_train['최대구매액']<= maxB['최대구매액']) & (x_train['최대구매액'] > x_train['최대구매액'].median())
conditionMaxBuySmall= x_train['최대구매액'] <= x_train['최대구매액'].median()
x_train.loc[conditionMaxBuyOutlier, '최대구매액많은편']=2
x_train.loc[conditionMaxBuy, '최대구매액많은편']=1
x_train.loc[conditionMaxBuySmall, '최대구매액많은편']=0

# display(x_train.head(), '---')
# display(x_train['최대구매액많은편'].value_counts()) # 2는 207개

## 메모리 관리. 변수 지우기
x_train.drop(columns='환불금액', inplace=True)

## 책- 상관관계
# display((x_train.corr() < -0.7) |(x_train.corr()>0.7))
# => (총구매액~최대구매액) , (최대구매액많은편~최대구매액) 의 상관관계가 0.7 초과이므로, 최대구매액을 없애자
x_train.drop(columns='최대구매액', inplace=True)
# display((x_train.corr() < -0.7) |(x_train.corr()>0.7)) # 더이상 다중공선성 문제있는 것은 없다
# display(x_train.head(), '---')

## 인코딩. 범주화
# display(x_train.columns[x_train.dtypes==object], '---') # ['주구매상품', '주구매지점']
# import sklearn # help(sklearn)
# from sklearn import preprocessing # dir(preprocessing) # LabelEncoder
from sklearn.preprocessing import LabelEncoder
# help(LabelEncoder) # examples #fit_transform(self, y)
encoder= LabelEncoder()
# print(type(encoder.fit_transform(x_train['주구매상품']))) # ndarray. inplace 없다
x_train['주구매상품']= encoder.fit_transform(x_train['주구매상품']) # astype 바로 못 붙인다
x_train['주구매지점']= encoder.fit_transform(x_train['주구매지점'])
x_train['주구매상품']= x_train['주구매상품'].astype('category') # inplace 없다
x_train['주구매지점']= x_train['주구매지점'].astype('category')
# 파생변수 범주화
x_train['주말방문여부']= x_train['주말방문여부'].astype('category')
x_train['환불여부']= x_train['환불여부'].astype('category')
x_train['최대구매액많은편']= x_train['최대구매액많은편'].astype('category')


# display(x_train.dtypes, x_train.head(), '---')

## cust_id 를 custid 로 변경하고 index 로 만들기
# print(y_train.head())
# y_train 은 cust_id 로 되어 있어서, 미리 변경하지 말고 to_csv 하기 직전에 바꾸자 <<
## 나중에 되돌리는 건 df.reset_index(drop=False, inplace=True)
# y_train 으로 보내야 하니 버릴 수는 없고, 
# help(x_train) # rename( . df.rename(columns={"A": "a", "B": "c"}, inplace=Ture)
# x_train.rename(columns={"cust_id":"custid"}, inplace=True) # 콜론!
# print(x_train.columns[0])
# print(x_train.columns[1::])
# print(x_train.columns, '---')
x_train.index= x_train['cust_id']
x_train.drop(columns='cust_id', inplace=True)

# display(x_train.head(), x_train.dtypes, '---')


## 스케일링 - 시험때는 조건말고 눈으로 보고 컬럼 골라내서 for 문 돌리자
# + 조건으로 거니까 앞에 코드에서 변경이 있어도, 최소 변경만으로 수월하게 진행가능
# from sklearn import preprocessing # dir(preprocessing) # gender는 범주형이니, StandardScaler
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()
x_trainNumCols= x_train.columns[(x_train.dtypes!=object) &(x_train.dtypes!='category')]
# print(x_trainNumCols)
# print(x_trainNumCols[x_train.describe().loc['max']> 10000])
bigNumCols= x_trainNumCols[x_train.describe().loc['max']> 30] # 30 근거: 범주컬럼 냅두고 싶어서 눈으로 정한 값
# scaler.fit_transform()
for col in bigNumCols:
    x_train[col]= scaler.fit_transform(x_train[[col]])
    
# display(x_train.head(), x_train.dtypes, '---')

beforeModeling= time.time()
print(f"모델링전까지 수행시간: {beforeModeling-start_time:.6} sec")

## 모델링
# import sklearn # help(sklearn) #package. ensemble
# from sklearn import ensemble # dir(ensemble) #RandomForestClassifier - gender
from sklearn.ensemble import RandomForestClassifier
model= RandomForestClassifier(n_estimators= 40) # help(model) #fit # 메모리 줄이기. n_estimators=100 default
# print(y_train.head()) # y_train 이 확률값이 아니다 ! 모델은 값으로 만들고 나중에 제출할 때만 _proba 해야!

## 메모리 이슈
# 에러! MemoryError: could not allocate 458752000 byte= 437MB. 여유분 약 3GB
# MemoryError: could not allocate 229376000 bytes= 218MB
# model.fit_transform(x_train, y_train) # RandomForestClassifier 에 fit_transform 이 없다.

model.fit(x_train, y_train) # <<<

# 틀림. proba는 제출할때. y_train_proba= model.predict_proba(x_train) 
# 여태 만진 게 train인데 왜 갑자기 test를. 굳이 train ㄴㄴ. 바로 test ㄱㄱ. y_train_predict= model.predict(x_train)
# 에러 y_test_predict= model.predict(x_test) # ValueError: could not convert string to float: '기타'

y_train_predict= model.predict(x_train) # <<<
y_test_predict= model.predict(x_test) # <<< train 으로 만든 모델에 x_test 넣기. roc_auc_score 뽑으려면 필수.
# 당연히 에러나겠지. 파생변수 없고 null 이 그득해서.

display(pd.DataFrame(y_train_predict).head()) # <<<

# print(model.score(y_train, y_train_predict)) # <<<
# Error: ValueError: X has 2 features, but DecisionTreeClassifier is expecting 10 features as input
# 랜포에 왠 의사결정나무.?? 따지지 말고 model.score 를 쓰지 말자.

# from sklearn import metrics # dir(metrics) # roc_auc_score
from sklearn.metrics import roc_auc_score
# help(roc_auc_score) #roc_auc_score(y, clf.predict_proba(X)[:, 1])
print(roc_auc_score(y_test, y_test_predict))


afterModeling= time.time()
print(f"모델링 시간: {afterModeling-beforeModeling:.6} sec")

모델링전까지 수행시간: 0.151001 sec


ValueError: could not convert string to float: '기타'

[Memory Error 해결방법](https://bskyvision.com/799)
114688000 byte(109 MB) 까지 줄였다
1. 재실행 및 재부팅 후 실행 -> 마찬가지
2. batch 사이즈 줄이기 -> n_estimators, max_depth 정도?
> n_estimators=40
3. 도중에 중단되도 메모리 해제하기 -> 파이썬에는 메모리 관리 명령어가 없다 (!!)
> [python try except finally memory leak](https://stackoverflow.com/a/60454634)
4. 변수를 줄인다 -> 파생변수를 지우거나 원본변수를 지우거나
> 최대구매금액, 환불금액 삭제
5. 내 PC에서 메모리를 먹고 있는 다른 서비스 종료
6. 페이징 파일을 증가시킨다 -> 내 PC에서는 가능하더라도, 시험장에서는 .?

In [3]:
from sklearn.ensemble import RandomForestClassifier
# help(RandomForestClassifier) 
#(n_estimators=100, *, criterion='gini', max_depth=None, min_samples_split=2, 
# min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, 
# min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, 
#n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, 
# max_samples=None)